In [1]:
# 1. Get 10,000 posts

In [1]:
import sys
import requests
calais_url = 'https://api.thomsonreuters.com/permid/calais'

import webhoseio, os
from gensim.models import KeyedVectors
import json
from scipy import spatial
from simhash import Simhash, SimhashIndex
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk, re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, sent_tokenize, ngrams
stopwords = set(nltk.corpus.stopwords.words('english'))

import spacy
import en_core_web_sm

In [209]:
webhoseio.config(token="2c45808b-852d-4156-af2f-faf960992ea9")
query_params = {"q": "organization:apple language:english", #change to other org to get 10,000 feeds
                "sort": "crawled"}
output = webhoseio.query("filterWebContent", query_params)
print(output['posts'][0]['text']) # Print the text of the first post
print(output['posts'][0]['published']) # Print the text of the first post publication date

# Print the site of the first post
print(output['posts'][0]['thread']['site'])

feeds = []
for item in output['posts']:
    feeds.append(item)
    #print(item['title'])
print(len(feeds))

count = 99
while count > 0:    
    output = webhoseio.get_next()
    for item in output['posts']:
        feeds.append(item)
    count -= 1
#     if len(feeds) > 10000:
#         break
# feeds = feeds[:10000]
print(len(feeds)) #try changed org to get 10,000 feeds

with open("anly540_final.json", "w") as myfile: #change file name to other org
    for feed in feeds:
        line = json.dumps(feed)
        myfile.write(line)
        myfile.write("\n")

favorite this post Apple Watch Series 3 - $275 hide this posting unhide < image 1 of 1 > ( google map ) QR Code Link to This Post I have a 42mm, series 3 Apple Watch with a Nike Band and stock band as well. If interested call or text me at show contact info do NOT contact me with unsolicited services or offers post id: 6768967027
2018-12-10T05:31:00.000+02:00
craigslist.org
100
7229


In [210]:
json_data = open("anly540_final.json").readlines()
feeds_read_from_file = []
for line in json_data:
    feeds_read_from_file.append(json.loads(line))
print(len(feeds_read_from_file))

for feed in feeds_read_from_file[:10]:
    print(feed['title'], '\n')
    print(feed['text'], '\n')
    print(feed['url'], '\n')
#print(feeds_read_from_file[0]['title'])

7229
Apple Watch Series 3 $275 

favorite this post Apple Watch Series 3 - $275 hide this posting unhide < image 1 of 1 > ( google map ) QR Code Link to This Post I have a 42mm, series 3 Apple Watch with a Nike Band and stock band as well. If interested call or text me at show contact info do NOT contact me with unsolicited services or offers post id: 6768967027 

http://omgili.com/ri/.wHSUbtEfZS.NqVwkWZ1TAqdv4urj.vjxgKu6anMqNNjLE2z0fZ9QnAmauH3wE_5w3.LJhgiMHyY.va_9iccLGK1bmL5NBGJqsxMabWV7ec- 

The stock market's severe drop could be an ominous sign 

So is this it — the end of the decade-long bull market? Or is it the sort of normal, healthy pullback that often follows a strong showing?
Last week, it felt like the bottom dropped out of the stock market.
The blue-chip Dow Jones industrial average fell 4.5 percent for the week, ending with a 559-point rout Friday that left the index in negative territory for the year.
The benchmark Standard & Poor's 500-stock index has fallen into a "cor

In [211]:
# 2. Deduplication

In [212]:
# Loading Word2Vec Google News model
model_path = './'
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec Google News', 'GoogleNews-vectors-negative300.bin', True)

Loading Word2Vec Google News model...
Finished loading Word2Vec Google News model...


In [213]:
# Original set of Titles and Publish dates
feeds = []
i = 0
for feed in feeds_read_from_file:
    feed['id'] = i
#     if feed['language'] == 'english':
#         print(feed['language'])    
#     print(str(feed['id']), str(feed['title']),  feed['published'])
    i += 1
    feeds.append(feed)

In [214]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [215]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [216]:
def cleanup(input):
    # remove English stopwords
    input = input.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
    input = re.sub(r'[^a-zA-Z0-9 ]', '', input)
    return input

In [217]:
# Clean identical titles
dups = set()
for i in range(len(feeds)):
    for j in range(i + 1, len(feeds)):
        if j not in dups and feeds[i]['title'].lower() == feeds[j]['title'].lower():
            dups.add(j)
#     print(len(dups))

In [218]:
# Get unique feeds for further deduplication process
feeds_unique = []
for i, feed in enumerate(feeds):
    if i not in dups:
        feeds_unique.append(feed)

In [219]:
for i, feed in enumerate(feeds_unique):
    feed['id'] = i
# feeds_unique
len(feeds_unique)

5279

In [220]:
# Set up similarity distance as 5 for further deduplication process
distance = 10
objs = [(str(feed['id']), Simhash(str(feed['title']))) for feed in feeds_unique]
index = SimhashIndex(objs, k=distance)
# index.bucket_size()

In [221]:
# indices of duplicated titles
dups = set()
for feed in feeds_unique:
    # Select a feed from the list based on its index
    SELECTED_INDEX = feed['id'] #27
#     if SELECTED_INDEX in dups:
#         continue
    feed_sel = feeds[SELECTED_INDEX]
    # calculate  hash value
    feed_hash = Simhash(str(feed_sel['title']))
    # find all duplicate indices
    dup_indices = index.get_near_dups(feed_hash)
    for dup_index in dup_indices:
        if dup_index != str(SELECTED_INDEX):
            dups.add(dup_index)
            print("Dup#" + str(dup_index) + ": " + feeds[int(dup_index)]['title'])
#     print(dups)
    if len(dup_indices) > 1: 
        print("Original#" + str(SELECTED_INDEX) + ": " + feed_sel['title'])
        print("Number of duplicates (SimHash): " + str(len(dup_indices) - 1))
        print("")

Big bucket found. key:2:2, len:268
Big bucket found. key:12:3, len:290
Big bucket found. key:1e:4, len:212
Big bucket found. key:3:5, len:217
Big bucket found. key:18:9, len:267
Big bucket found. key:13:1, len:274
Big bucket found. key:1:8, len:206
Big bucket found. key:3:1, len:243
Big bucket found. key:1a:2, len:202
Big bucket found. key:11:3, len:211
Big bucket found. key:3:6, len:275
Big bucket found. key:11:1, len:233
Big bucket found. key:10:2, len:290
Big bucket found. key:18:3, len:264
Big bucket found. key:1e:4, len:212
Big bucket found. key:b:5, len:243
Big bucket found. key:17:7, len:217
Big bucket found. key:7:8, len:210
Big bucket found. key:1f:5, len:251
Big bucket found. key:11:1, len:233
Big bucket found. key:1e:4, len:212
Big bucket found. key:0:8, len:206
Big bucket found. key:a:9, len:205
Big bucket found. key:16:3, len:204
Big bucket found. key:1:8, len:206
Big bucket found. key:a:9, len:205
Big bucket found. key:1:1, len:243
Big bucket found. key:12:3, len:290
Big 

Dup#2723: The Sound is Coming from the Streets
Dup#2120: Monday Morning Reads
Dup#2290: The action in Apple's stock tells you everything you need to know about this market, says Jim Cramer
Dup#5117: Supermicro concludes 'Big Hack' investigation, says no tampering - CyberScoop
Dup#4901: Brian Watt: The Desecration of a Cathedral in Vienna
Dup#3697: The Galaxy S10 Will Have a Headphone Jack, Turning It Into a Luxury Feature
Dup#982: Apple is losing the interest of some big-money investors, RBC says
Original#0: Apple Watch Series 3 $275
Number of duplicates (SimHash): 7

Dup#1046: Apple will continue iPhone sales in China, despite Qualcomm battle
Original#5: Family vs screen time: can we get our kids back?
Number of duplicates (SimHash): 1

Dup#703: Certified Sustainability Practitioner Program, Advanced Edition 2018
Original#6: Is Silicon Valley Morphing Into The Morality Police?
Number of duplicates (SimHash): 1



Big bucket found. key:1:8, len:206
Big bucket found. key:0:9, len:314
Big bucket found. key:1:1, len:243
Big bucket found. key:14:3, len:223
Big bucket found. key:6:6, len:255
Big bucket found. key:1:8, len:206
Big bucket found. key:10:0, len:264
Big bucket found. key:18:2, len:210
Big bucket found. key:18:2, len:210
Big bucket found. key:10:3, len:324
Big bucket found. key:1d:5, len:202
Big bucket found. key:16:7, len:232
Big bucket found. key:4:8, len:207
Big bucket found. key:8:9, len:527
Big bucket found. key:1:1, len:243
Big bucket found. key:13:2, len:215
Big bucket found. key:8:3, len:213
Big bucket found. key:1e:4, len:212
Big bucket found. key:11:3, len:211
Big bucket found. key:1:0, len:288
Big bucket found. key:b:9, len:207
Big bucket found. key:14:0, len:214
Big bucket found. key:12:2, len:272
Big bucket found. key:7:1, len:234
Big bucket found. key:7:8, len:210
Big bucket found. key:1:9, len:285
Big bucket found. key:1a:3, len:208
Big bucket found. key:8:5, len:215
Big buc

Dup#4279: [New post] Get ripped for FREE with this rare price drop on PSHPS for iOS (Reg. $1)
Original#8: Apple’s $29 iPhone Battery Replacement Program Ends December 31
Number of duplicates (SimHash): 1



Big bucket found. key:1:6, len:216
Big bucket found. key:4:0, len:218
Big bucket found. key:3:1, len:243
Big bucket found. key:13:1, len:274
Big bucket found. key:1d:5, len:202
Big bucket found. key:12:3, len:290
Big bucket found. key:0:6, len:242
Big bucket found. key:7:8, len:210
Big bucket found. key:1:1, len:243
Big bucket found. key:15:7, len:226
Big bucket found. key:b:8, len:203
Big bucket found. key:0:0, len:364
Big bucket found. key:13:1, len:274
Big bucket found. key:10:3, len:324
Big bucket found. key:f:8, len:203
Big bucket found. key:1:0, len:288
Big bucket found. key:1:1, len:243
Big bucket found. key:8:2, len:253
Big bucket found. key:9:5, len:245
Big bucket found. key:17:1, len:238
Big bucket found. key:3:2, len:219
Big bucket found. key:4:7, len:215
Big bucket found. key:4:8, len:207
Big bucket found. key:3:1, len:243
Big bucket found. key:3:8, len:207
Big bucket found. key:8:9, len:527
Big bucket found. key:18:9, len:267
Big bucket found. key:18:3, len:264
Big bucket 

...

Dup#3742: The state Duma is considering tightening control over Apple Pay and Android Pay
Dup#3743: Released the first beta of the new operating systems Apple
Dup#3744: The Huawei arrest puts a 'bullseye' on Apple's back at a potentially dangerous time for iPhone sales in China
Dup#3744: The Huawei arrest puts a 'bullseye' on Apple's back at a potentially dangerous time for iPhone sales in China
Dup#3876: Tip: take faster heart rate readings with higher fidelity on Apple Watch Series 4
Dup#3745: Apple Hit With Sales Ban on Older iPhones in China
Dup#4223: How To Add the Google Calendar App to Your iPhone
Original#5261: Apple's China headache worsens as court bans iPhone sales
Number of duplicates (SimHash): 2

Dup#3746: China blocks iPhone sales over patent dispute
Dup#3747: Apple Pay finally launches in Germany Via TechCrunch
Dup#3748: S o T o S p e a k | Ep. 107 | Phone Wars


Big bucket found. key:b:9, len:207
Big bucket found. key:19:3, len:208
Big bucket found. key:f:5, len:228
Big bucket found. key:b:8, len:203
Big bucket found. key:13:2, len:215
Big bucket found. key:3:6, len:275
Big bucket found. key:1c:7, len:209
Big bucket found. key:19:3, len:208
Big bucket found. key:f:5, len:228
Big bucket found. key:b:8, len:203
Big bucket found. key:3:2, len:219
Big bucket found. key:6:6, len:255
Big bucket found. key:13:1, len:274
Big bucket found. key:12:3, len:290
Big bucket found. key:8:2, len:253
Big bucket found. key:9:5, len:245
Big bucket found. key:1:8, len:206
Big bucket found. key:1a:3, len:208
Big bucket found. key:1d:5, len:202
Big bucket found. key:12:2, len:272


Dup#3749: Howard Lindzon - Investor, Entrepreneur - Investing for Profit and Joy
Dup#3750: Video in this thread Ho Azz Samsung Kills Headphone Jack in Latest Smartphone After Years of Mocking Apple
Dup#3751: iPad Pro 12.9" WiFi 32gb (Piedmont) $450
Dup#3750: Video in this thread Ho Azz Samsung Kills Headphone Jack in Latest Smartphone After Years of Mocking Apple
Dup#3752: Samsung Galaxy S10 Lite concept render shows even bezels, no chin
Dup#3753: [New post] Amazon’s popular 12-inch MacBook refurb sale is back at $820 shipped
Dup#3754: Move over notch, the hole-punch smartphone camera is coming
Dup#3755: After Sharp Reversal Monday, Positive Momentum Carries Over On China, FAANGs


Big bucket found. key:3:6, len:275
Big bucket found. key:13:2, len:215
Big bucket found. key:3:6, len:275
Big bucket found. key:1c:7, len:209
Big bucket found. key:18:2, len:210
Big bucket found. key:1a:3, len:208
Big bucket found. key:f:8, len:203
Big bucket found. key:5:0, len:214
Big bucket found. key:13:2, len:215
Big bucket found. key:9:9, len:318
Big bucket found. key:13:1, len:274
Big bucket found. key:10:2, len:290
Big bucket found. key:10:3, len:324
Big bucket found. key:d:5, len:228
Big bucket found. key:16:6, len:201


Dup#3756: Immediate joiner IOS Developer - Linosys Solutions Private Limited - Pune, Maharashtra
Dup#3757: What You Need to Know About Apple's Capital Return Program
Dup#3751: iPad Pro 12.9" WiFi 32gb (Piedmont) $450
Dup#3758: Court orders China sales ban on older Apple iPhones in Qualcomm case
Dup#3748: S o T o S p e a k | Ep. 107 | Phone Wars
Dup#3759: Here's What's Going On in the Wearables Market


In [222]:
# Clean up deduplicated dataset
feeds_unique_deduped = []
for i, feed in enumerate(feeds_unique):
    if str(feed['id']) not in dups:
        feeds_unique_deduped.append(feed)
len(feeds_unique_deduped)
len(dups)

3751

In [223]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is np.nan:
        result = 0
        
    return result

In [224]:
count_dup = 0
for dupi in dup_indices:
    try:
        score = vec_similarity(feed_sel['title'], feeds[int(dupi)]['title'], model_word2vec)
#         print(score)
    except:
        score = 0
    if score > 0.9:
        count_dup += 1
        print(feeds[int(dupi)]['title'])
print("Number of Dupies (SimHash + WordVectors): " + str(count_dup))

Number of Dupies (SimHash + WordVectors): 0


In [225]:
# Save as txt file and print deduplicated subset of id, titles and published dates
# dup_indices_set = set(dup_indices)
with open('./deduped.txt', 'w') as f:
    for feed in feeds:
        if str(feed['id']) not in dups:
#             print(' '.join((str(feed['id']), str(feed['title']),  feed['published'])))
            f.write(' '.join((str(feed['id']), str(feed['title']),  feed['published'], '\n')))

In [226]:
#3. Named Entity Recognition

In [227]:
def sendFile(input_data, headers, output_dir='./'):
#     with open(file_name, 'rb') as input_data:
    response = requests.post(calais_url, data=input_data, headers=headers, timeout=80)
    print ('status code: %s' % response.status_code)
    content = response.text
#     print ('Results received: %s' % content)
    if response.status_code == 200:
        saveFile('final_text', output_dir, content)
        return response
            
def saveFile(file_name, output_dir, content):
    output_file_name = os.path.basename(file_name) + '.json'
    output_file = open(os.path.join(output_dir, output_file_name), 'wb')
    output_file.write(content.encode('utf-8'))
    output_file.close()

In [228]:
feeds_unique_deduped
RANDOM_INDEX = 1337
print(feeds_unique_deduped[RANDOM_INDEX]['title'])
access_token = 'ZR2BvCL0pxKthAeKa0FKPbb9I9At9ows'
headers = {'X-AG-Access-Token' : access_token, 
           'Content-Type' : 'text/raw', 
           'outputformat' : 'application/json'
          }
input_data = feeds_unique_deduped[RANDOM_INDEX]['title'].encode('utf-8')
response = sendFile(input_data, headers)
tagging_result = json.loads(response.content)

Apple iPhone 7 128 gig (Rutland) $250
status code: 200


In [229]:
def extract_entity(tagging_result):
    for hash in tagging_result:        
        if '_typeGroup' in tagging_result[hash] and tagging_result[hash]['_typeGroup'] == 'entities':
#             print(tagging_result[hash])
            print(tagging_result[hash]['_type'], ':', 
                  tagging_result[hash]['name'], ', relevance: ', 
                  tagging_result[hash]['relevance'])
extract_entity(tagging_result)

Company : APPLE , relevance:  0
Product : Apple iPhone , relevance:  0.8


In [231]:
#4. Topic Clusters
# feeds_unique_deduped

In [232]:
def tokenize_titles(title):
    tokens = nltk.word_tokenize(title)
    lmtzr = WordNetLemmatizer()
    filtered_tokens = []
    
    for token in tokens:
        token = token.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
        token = re.sub(r'[^a-zA-Z0-9 ]', '', token)
        if token not in stopwords:
            filtered_tokens.append(token.lower())
    
    lemmas = [lmtzr.lemmatize(t,'v') for t in filtered_tokens]

    return lemmas

In [239]:
def clstr_lda(num_topics, titles):
    # top words to be identified
    n_top_words = 10

    tf_vectorizer = CountVectorizer(max_df=0.96, min_df=0.02, max_features=1000,
                                    tokenizer=tokenize_titles, ngram_range=(3,4))

    tf = tf_vectorizer.fit_transform(titles)

    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=1000,
                                    learning_method='batch', learning_offset=10.,
                                    random_state = 1)
    lda.fit(tf)
    tf_feature_names = tf_vectorizer.get_feature_names()

    # print top topic words
    topics = dict()
    for topic_idx, topic in enumerate(lda.components_):
        topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print("Topic #%d:" % topic_idx)
        print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
    return topics

In [248]:
feed_titles = []
count = 0
for feed in feeds_unique_deduped:
    if 'watch' not in feed['title'].lower():
#         count += 1
#         print(feed['title'])
        feed_titles.append(str(feed['title']))
print(count)
print("Total number of titles: " + str(len(feed_titles)))

0
Total number of titles: 1434


In [251]:
feed_titles[:200]

["The stock market's severe drop could be an ominous sign",
 'Global Smart Machines Market Growth and Forecast By Top Players | KUKA, IBM, Apple, Google, Aethon, Cerner, Microsoft, Philips Healthcare',
 'Brand New IMAC PRO (Boone NC) $4000',
 "Facebook slips from 1st to 7th in 2019 'Best Places to Work' list",
 'Apple’s $29 iPhone Battery Replacement Program Ends December 31',
 'The Fall of the iPhone',
 'Look how Apple and Google use copy to differentiate themselves from the crowd.',
 "4 Facts About Apple's Fastest-Growing Catalyst",
 'It hasn’t been a great week for tech giants, a few of whom w',
 'TEST DRIVE: 2019 BMW i3 120Ah',
 'Huawei has surpassed Apple as the world’s second largest smartphone brand',
 'The Digital Transformation',
 '(Red) campaign is an effective way to raise money for charity',
 'Бесконтактная оплата проезда внедрена в Добрянке',
 'Apple reveals $150 smart mattress sensor that monitors your sleep',
 'The arrest of Huawei’s CFO by Western law enforcers',
 '15" 

In [250]:
topics = clstr_lda(5, feed_titles)

Topic #0:
   |  apple 
Topic #1:
 apple  |   
Topic #2:
   |  apple 
Topic #3:
   |  apple 
Topic #4:
   |  apple 


In [ ]:
#5. Topic Taxonomy

In [252]:
nlp = en_core_web_sm.load()
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
OBJECTS = ["dobj", "dative", "attr", "oprd"] 

In [274]:
for feed in feeds_unique_deduped[:1]:
    selected_feed = feed['text']
#     print(selected_feed) 
    sentences = sent_tokenize(selected_feed)
    for sentence in sentences:
        print(sentence)
        print()

So is this it — the end of the decade-long bull market?

Or is it the sort of normal, healthy pullback that often follows a strong showing?

Last week, it felt like the bottom dropped out of the stock market.

The blue-chip Dow Jones industrial average fell 4.5 percent for the week, ending with a 559-point rout Friday that left the index in negative territory for the year.

The benchmark Standard & Poor's 500-stock index has fallen into a "correction," meaning it's off more than 10 percent from its peak reached Sept. 20.

And once-highflying tech stocks faltered with Apple, Facebook and Google parent Alphabet all ending last week showing stock-price losses for the year — driving the Nasdaq composite index into a correction too.

So is this it — the end of the decade-long bull market?

Or is it the sort of normal, healthy pullback that often follows a strong showing?

"Everyone is trying to figure that out," said Sam Stovall, chief investment strategist at CFRA Research, though he's amo

In [275]:
def getSubsFromConjunctions(subs):
    moreSubs = []
    for sub in subs:
        # rights is a generator
        rights = list(sub.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreSubs.extend([tok for tok in rights if tok.dep_ in SUBJECTS or tok.pos_ == "NOUN"])
            if len(moreSubs) > 0:
                moreSubs.extend(getSubsFromConjunctions(moreSubs))
    return moreSubs

In [276]:
def getObjsFromConjunctions(objs):
    moreObjs = []
    for obj in objs:
        # rights is a generator
        rights = list(obj.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreObjs.extend([tok for tok in rights if tok.dep_ in OBJECTS or tok.pos_ == "NOUN"])
            if len(moreObjs) > 0:
                moreObjs.extend(getObjsFromConjunctions(moreObjs))
    return moreObjs

In [277]:
def getVerbsFromConjunctions(verbs):
    moreVerbs = []
    for verb in verbs:
        rightDeps = {tok.lower_ for tok in verb.rights}
        if "and" in rightDeps:
            moreVerbs.extend([tok for tok in verb.rights if tok.pos_ == "VERB"])
            if len(moreVerbs) > 0:
                moreVerbs.extend(getVerbsFromConjunctions(moreVerbs))
    return moreVerbs

In [278]:
def findSubs(tok):
    head = tok.head
    while head.pos_ != "VERB" and head.pos_ != "NOUN" and head.head != head:
        head = head.head
    if head.pos_ == "VERB":
        subs = [tok for tok in head.lefts if tok.dep_ == "SUB"]
        if len(subs) > 0:
            verbNegated = isNegated(head)
            subs.extend(getSubsFromConjunctions(subs))
            return subs, verbNegated
        elif head.head != head:
            return findSubs(head)
    elif head.pos_ == "NOUN":
        return [head], isNegated(tok)
    return [], False

In [279]:
def isNegated(tok):
    negations = {"no", "not", "n't", "never", "none"}
    for dep in list(tok.lefts) + list(tok.rights):
        if dep.lower_ in negations:
            return True
    return False

In [280]:
def findSVs(tokens):
    svs = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        if len(subs) > 0:
            for sub in subs:
                svs.append((sub.orth_, "!" + v.orth_ if verbNegated else v.orth_))
    return svs

In [281]:
def getObjsFromPrepositions(deps):
    objs = []
    for dep in deps:
        if dep.pos_ == "ADP" and dep.dep_ == "prep":
            objs.extend([tok for tok in dep.rights if tok.dep_  in OBJECTS or (tok.pos_ == "PRON" and tok.lower_ == "me")])
    return objs

In [282]:
def getObjsFromAttrs(deps):
    for dep in deps:
        if dep.pos_ == "NOUN" and dep.dep_ == "attr":
            verbs = [tok for tok in dep.rights if tok.pos_ == "VERB"]
            if len(verbs) > 0:
                for v in verbs:
                    rights = list(v.rights)
                    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
                    objs.extend(getObjsFromPrepositions(rights))
                    if len(objs) > 0:
                        return v, objs
    return None, None

In [283]:
def getObjFromXComp(deps):
    for dep in deps:
        if dep.pos_ == "VERB" and dep.dep_ == "xcomp":
            v = dep
            rights = list(v.rights)
            objs = [tok for tok in rights if tok.dep_ in OBJECTS]
            objs.extend(getObjsFromPrepositions(rights))
            if len(objs) > 0:
                return v, objs
    return None, None

In [284]:
def getAllSubs(v):
    verbNegated = isNegated(v)
    subs = [tok for tok in v.lefts if tok.dep_ in SUBJECTS and tok.pos_ != "DET"]
    if len(subs) > 0:
        subs.extend(getSubsFromConjunctions(subs))
    else:
        foundSubs, verbNegated = findSubs(v)
        subs.extend(foundSubs)
    return subs, verbNegated

In [285]:
def getAllObjs(v):
    # rights is a generator
    rights = list(v.rights)
    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
    objs.extend(getObjsFromPrepositions(rights))

    #potentialNewVerb, potentialNewObjs = getObjsFromAttrs(rights)
    #if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
    #    objs.extend(potentialNewObjs)
    #    v = potentialNewVerb

    potentialNewVerb, potentialNewObjs = getObjFromXComp(rights)
    if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
        objs.extend(potentialNewObjs)
        v = potentialNewVerb
    if len(objs) > 0:
        objs.extend(getObjsFromConjunctions(objs))
    return v, objs

In [286]:
def findSVOs(tokens):
    svos = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        # hopefully there are subs, if not, don't examine this verb any longer
        if len(subs) > 0:
            v, objs = getAllObjs(v)
            for sub in subs:
                for obj in objs:
                    objNegated = isNegated(obj)
                    svos.append((sub.lower_, "!" + v.lower_ if verbNegated or objNegated else v.lower_, obj.lower_))
    return svos

In [287]:
def getAbuserOntoVictimSVOs(tokens):
    maleAbuser = {'he', 'boyfriend', 'bf', 'father', 'dad', 'husband', 'brother', 'man'}
    femaleAbuser = {'she', 'girlfriend', 'gf', 'mother', 'mom', 'wife', 'sister', 'woman'}
    neutralAbuser = {'pastor', 'abuser', 'offender', 'ex', 'x', 'lover', 'church', 'they'}
    victim = {'me', 'sister', 'brother', 'child', 'kid', 'baby', 'friend', 'her', 'him', 'man', 'woman'}

    svos = findSVOs(tokens)
    wnl = WordNetLemmatizer()
    passed = []
    for s, v, o in svos:
        s = wnl.lemmatize(s)
        v = "!" + wnl.lemmatize(v[1:], 'v') if v[0] == "!" else wnl.lemmatize(v, 'v')
        o = "!" + wnl.lemmatize(o[1:]) if o[0] == "!" else wnl.lemmatize(o)
        if s in maleAbuser.union(femaleAbuser).union(neutralAbuser) and o in victim:
            passed.append((s, v, o))
    return passed

In [288]:
def printDeps(toks):
    for tok in toks:
        print(tok.orth_, tok.dep_, tok.pos_, tok.head.orth_, [t.orth_ for t in tok.lefts], [t.orth_ for t in tok.rights])

def testSVOs(sentences):
    #nlp = English()
    for sentence in sentences:
        tok = nlp(sentence)
        svos = findSVOs(tok)
    #     printDeps(tok)
    #    assert set(svos) == {('i', '!have', 'assistance'), ('he', '!provide', 'support')}
        if svos:
            for svo in svos:
                print('S:{0}, V:{1}, O:{2}'.format((svo[0]), svo[1], svo[2]))
        else:
            print('No SVO')

In [289]:
testSVOs()

No SVO
S:that, V:follows, O:showing
No SVO
S:that, V:left, O:index
S:it, V:'s, O:percent
S:stocks, V:faltered, O:all
No SVO
S:that, V:follows, O:showing
S:everyone, V:figure, O:that
S:obscuring, V:are, O:trends
S:that, V:keeping, O:pressure
S:trends, V:sparking, O:trading
S:they, V:include, O:tariffs
S:they, V:include, O:battle
S:they, V:include, O:prices
S:they, V:include, O:bursts
S:prices, V:raising, O:rates
S:pullback, V:is, O:—
No SVO
No SVO
No SVO
S:it, V:was, O:loss
S:market, V:reached, O:levels
S:s&p, V:gained, O:percent
No SVO
No SVO
S:we, V:had, O:amount
S:year, V:!was, O:anomaly
No SVO
No SVO
S:s&p, V:lost, O:percent
S:s&p, V:lost, O:decline
No SVO
S:yields, V:locking, O:money
No SVO
No SVO
S:investors, V:pay, O:attention
No SVO
S:it, V:be, O:buy
S:s&p, V:trading, O:earnings
No SVO
No SVO
S:street, V:expected, O:growth
S:we, V:see, O:amount
No SVO


In [ ]:
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

# model_word2vec = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin', True)
# model_fasttext = load_wordvec_model('FastText', 'fastText_wiki_en.vec', False)

In [ ]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [ ]:
# function takes an input string, runs similarity for each item in topic_taxonomy, sorts and returns top 3 results
def classify_topics(input, vectors):
    feed_score = dict()
    for key, value in topic_taxonomy.items():
        max_value_score = dict()
        for label, keywords in value.items():
            max_value_score[label] = 0
            topic = (key + ' ' + keywords).strip()
            max_value_score[label] += float(calc_similarity(input, topic, vectors))
            
        sorted_max_score = sorted(max_value_score.items(), key=operator.itemgetter(1), reverse=True)[0]
        feed_score[sorted_max_score[0]] = sorted_max_score[1]
    return sorted(feed_score.items(), key=operator.itemgetter(1), reverse=True)[:3]

In [ ]:
# example using Word2Vec
output1 = classify_topics('White House Has Plan To Force Tillerson Out, Replace With CIA Chief', model_word2vec)
print(output1)
# # example output using FastText, note for FastText, string needs to be non-capitalized
# output2 = classify_topics('White House Has Plan To Force Tillerson Out Replace With CIA Chief'.lower(), model_fasttext)
# print(output2)
    